In [ ]:
import os
import time
import pylab
import torch
import pickle
import numpy as np
from math import *
import matplotlib as mpl
from cycler import cycler
from numpy.linalg import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
class Agent:
    def __init__(self, agent_id, speed, area_width, area_height):
        self.id = agent_id
        self.pos = np.array([
            np.random.uniform(0, area_width),
            np.random.uniform(0, area_height)
        ], dtype=np.float64)

        theta0 = np.random.uniform(-np.pi, np.pi)
        self.vel = vel_from_angle(theta0, speed)
        self.is_alive = 1

    def update_position(self, dt):
        if self.is_alive:
            self.pos += self.vel * dt


def angle_from_vel(v):
    return float(np.arctan2(v[1], v[0]))


def vel_from_angle(theta, speed):
    return np.array([np.cos(theta), np.sin(theta)], dtype=np.float64) * speed


def wrap_pi(theta):
    return (theta + np.pi) % (2 * np.pi) - np.pi


def enforce_walls(agent, area_width, area_height):
    if agent.pos[0] < 0:
        agent.pos[0] = 0
        agent.vel[0] *= -1
    elif agent.pos[0] > area_width:
        agent.pos[0] = area_width
        agent.vel[0] *= -1

    if agent.pos[1] < 0:
        agent.pos[1] = 0
        agent.vel[1] *= -1
    elif agent.pos[1] > area_height:
        agent.pos[1] = area_height
        agent.vel[1] *= -1

In [ ]:
def get_sim_features(prey_step, predator_step,
                     area_width=2160, area_height=2160,
                     max_speed=15.0, max_speed_env=25.0):

    prey_step = np.asarray(prey_step, dtype=np.float32)
    predator_step = np.asarray(predator_step, dtype=np.float32)

    data = np.concatenate([predator_step, prey_step], axis=0)
    px, py = data[:, 0], data[:, 1]
    vx, vy = data[:, 2], data[:, 3]

    xs = np.clip(px / area_width, 0.0, 1.0)
    ys = np.clip(py / area_height, 0.0, 1.0)

    thetas = np.arctan2(vy, vx)
    directions = thetas / (2 * np.pi) + 0.5
    thetas_norm = thetas / np.pi

    speed_raw = np.sqrt(vx**2 + vy**2)
    speeds_norm = np.clip(speed_raw / max_speed, 0.0, 1.0)
    speeds = speeds_norm * max_speed

    cos_t, sin_t = np.cos(thetas), np.sin(thetas)
    vxs = cos_t * speeds
    vys = sin_t * speeds

    dx = xs[None, :] - xs[:, None]
    dy = ys[None, :] - ys[:, None]

    rel_vx = cos_t[:, None] * vxs[None, :] + sin_t[:, None] * vys[None, :]
    rel_vy = -sin_t[:, None] * vxs[None, :] + cos_t[:, None] * vys[None, :]

    rel_vx = np.clip(rel_vx, -max_speed_env, max_speed_env) / max_speed_env
    rel_vy = np.clip(rel_vy, -max_speed_env, max_speed_env) / max_speed_env

    N = xs.shape[0]
    thetas_mat = np.tile(thetas_norm[:, None], (1, N))

    features = np.stack([dx, dy, rel_vx, rel_vy, thetas_mat], axis=-1)
    neigh = features[~np.eye(N, dtype=bool)].reshape(N, N - 1, 5)

    pred_tensor = torch.from_numpy(neigh[0]).unsqueeze(0)
    prey_tensor = torch.from_numpy(neigh[1:])

    return pred_tensor, prey_tensor, xs, ys, directions, dx, dy, vxs, vys

In [ ]:
def run_simulation(visualization='on', n=32, number_of_sharks=1, max_steps=1000, dt=0.5, constant_speed=50, shark_speed=50, area_width=2160, area_height=2160, theta_max=np.pi / 8 ):

    swarm  = [Agent(i, constant_speed, area_width, area_height) for i in range(n)]
    sharks = [Agent(i, shark_speed, area_width, area_height) for i in range(number_of_sharks)]

    prey_log = np.zeros((max_steps, n, 6), dtype=np.float32)
    predator_log = np.zeros((max_steps, number_of_sharks, 6), dtype=np.float32)

    pred_tensor, prey_tensor, xs, ys, directions, dx, dy, vxs, vys = get_sim_features(prey_step, predator_step, area_width=2160, area_height=2160, max_speed=15.0, max_speed_env=25.0)

    if visualization == 'on':
        _, ax = plt.subplots()

    for t in range(max_steps):

        # --------------------------------------------------------------
        # Logging
        # --------------------------------------------------------------
        for i, agent in enumerate(swarm):
            vnorm = norm(agent.vel)
            dir_xy = agent.vel / vnorm if vnorm > 0 else np.zeros(2)

            prey_log[t, i, 0:2] = agent.pos
            prey_log[t, i, 2:4] = agent.vel
            prey_log[t, i, 4:6] = dir_xy

        for j, shark in enumerate(sharks):
            vnorm = norm(shark.vel)
            dir_xy = shark.vel / vnorm if vnorm > 0 else np.zeros(2)

            predator_log[t, j, 0:2] = shark.pos
            predator_log[t, j, 2:4] = shark.vel
            predator_log[t, j, 4:6] = dir_xy

        # --------------------------------------------------------------
        # Visualization
        # --------------------------------------------------------------
        if visualization == 'on':
            ax.clear()
            ax.quiver(
                prey_log[t, :, 0],
                prey_log[t, :, 1],
                prey_log[t, :, 2],
                prey_log[t, :, 3],
            )
            ax.plot(predator_log[t, :, 0], predator_log[t, :, 1], 'ro')
            ax.set_xlim(0, area_width)
            ax.set_ylim(0, area_height)
            ax.set_aspect('equal')
            plt.pause(0.001)

        # ==============================================================
        # RANDOM HEADING UPDATE
        # ==============================================================
        for agent in swarm:
            if not agent.is_alive:
                continue

            theta_curr = angle_from_vel(agent.vel)
            dtheta = np.random.uniform(-theta_max, theta_max)
            theta_next = wrap_pi(theta_curr + dtheta)

            agent.vel = vel_from_angle(theta_next, constant_speed)

        for shark in sharks:
            if not shark.is_alive:
                continue

            theta_curr = angle_from_vel(shark.vel)
            dtheta = np.random.uniform(-theta_max, theta_max)
            theta_next = wrap_pi(theta_curr + dtheta)

            shark.vel = vel_from_angle(theta_next, shark_speed)

        # --------------------------------------------------------------
        # Physics
        # --------------------------------------------------------------
        for agent in swarm:
            enforce_walls(agent, area_width, area_height)
            agent.update_position(dt)

        for shark in sharks:
            enforce_walls(shark, area_width, area_height)
            shark.update_position(dt)


In [ ]:
mpl.use('TkAgg')
visualization = 'on'   # 'on' / 'off'
max_steps = 200       # maximum simulation steps
number_of_sharks = 1
number_of_prey = 32
prey_speed = 15
pred_speed = 15
area_width=50 #2160
area_height=50 #2160 
area_depth=50 


run_simulation(
    visualization='on',
    n=32,
    number_of_sharks=1,
    max_steps=200,
    dt=0.5,
    constant_speed=50,
    shark_speed=50,
    area_width=2160,
    area_height=2160,
    theta_max=np.pi / 8)